In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math


In [2]:
#load database

dataset = pd.read_csv('DiffData_classification_noise.csv')#.dropna()

# print (dataset.head(20)) #head of the data file with 20 rows
# print (dataset.tail(20))
print (dataset.describe().T) #simple statistics like mean, std, percentile

# edit database as needed
# reduce set of importance, rescale values etc...
df = dataset.copy()
df["target"] = [1 if label == "Dgb" else 0 for label in df["Diffusion_type"]]

# setup xdata which is input and y data which is target
#xInput = df.drop(["target"], axis = 1)
xInput = df[["distance","time"]]
yTarget = df[["target"]]

# print(xInput.head(10))
# print(yTarget.head(10))


          count        mean         std       min         25%         50%  \
time      980.0  300.000000  173.647540  0.000000  149.693252  300.000000   
distance  980.0    0.028601    0.016163 -0.000355    0.015133    0.026372   

                 75%         max  
time      450.306748  600.000000  
distance    0.041898    0.064679  


In [3]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np

# Split data for training and testing (replace xInput and yTarget with actual variable names)
X_train, X_test, y_train, y_test = train_test_split(xInput, yTarget, test_size=0.2, random_state=0)

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Set the number of neighbors for KNN
k = 5  # Adjust `k` as needed for optimization

# Initialize and train the KNN model
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Print evaluation metrics
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Get cross-validation score
scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5)
print("\nCross-Validation Scores:", scores)
print("Final CV Score: {:.2f}".format(np.mean(scores)))



Confusion Matrix:
 [[86 12]
 [ 4 94]]

Accuracy: 0.9183673469387755

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.88      0.91        98
           1       0.89      0.96      0.92        98

    accuracy                           0.92       196
   macro avg       0.92      0.92      0.92       196
weighted avg       0.92      0.92      0.92       196


Cross-Validation Scores: [0.92356688 0.92356688 0.94267516 0.9044586  0.91025641]
Final CV Score: 0.92


In [4]:
# split data for training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xInput,yTarget, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

from sklearn.svm import SVC
# Create a baseline SVM classifier
baseline_svm = SVC()
baseline_svm.fit(x_train, y_train.values.ravel())
y_pred = baseline_svm.predict(x_test)

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Define the hyperparameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Initialize the KNeighborsClassifier model
knn = KNeighborsClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, verbose=1)

# Fit the model with the grid of hyperparameters
grid_search.fit(X_train, y_train.values.ravel())


Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             verbose=1)

In [6]:
from sklearn.metrics import accuracy_score

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred_best = best_model.predict(x_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

print(f"Best SVM Accuracy: {accuracy_best:.2f}")
print(f"Best Hyperparameters: {best_params}")


Best SVM Accuracy: 0.93
Best Hyperparameters: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}
